In [1]:
%load_ext autoreload

In [5]:
# !pip3 install -r requirements.txt

In [1]:
from einops import rearrange
from glob import glob
import torch
from PIL import Image
from pathlib import Path

from models.perciever import Perceiver
from models.pilotnet import PilotNet
from data_prep.nvidia import NvidiaDataset

In [2]:
dataset_path = Path('/gpfs/space/projects/rally2023/rally-estonia-cropped-antialias')
paths = [
    dataset_path / '2021-05-20-12-36-10_e2e_sulaoja_20_30',
    dataset_path / '2021-05-20-12-43-17_e2e_sulaoja_20_30'
]

BATCH_SIZE=32
NUM_WORKERS=8

dataset = NvidiaDataset(paths)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True,
                                                   num_workers=NUM_WORKERS, pin_memory=True,
                                                   persistent_workers=True, collate_fn=dataset.collate_fn)

[NvidiaDataset] Using default transform: Compose(
)


FileNotFoundError: [Errno 2] No such file or directory: '\\gpfs\\space\\projects\\rally2023\\rally-estonia-cropped-antialias\\2021-05-20-12-36-10_e2e_sulaoja_20_30\\nvidia_frames.csv'

In [3]:
dataset_path = 'data_stuff/rally-estonia-cropped-antialias'
#dataset_path = '/gpfs/space/projects/rally2023/rally-estonia-cropped-antialias'
dataset_subdirs = glob(dataset_path + '/*')
dataset_subdir = dataset_subdirs[0]
print(dataset_subdir)
image_paths = glob(dataset_subdir + '/front_wide/*.png')
image = Image.open(image_paths[0])
pixels = list(image.getdata())
print(len(pixels))
pixel_tens = torch.Tensor(pixels)
print(pixel_tens.shape)
pixel_tens = pixel_tens.view(68, 264, 3)
print(pixel_tens.shape)
pixel_tens = rearrange(pixel_tens, 'h w c -> c h w')
print(pixel_tens.shape)




data_stuff/rally-estonia-cropped-antialias\2021-05-20-12-36-10_e2e_sulaoja_20_30
17952
torch.Size([17952, 3])
torch.Size([68, 264, 3])
torch.Size([3, 68, 264])


In [4]:
pixel_tens
batch = pixel_tens.view(1, *pixel_tens.shape)
print(batch.size())

torch.Size([1, 3, 68, 264])


In [13]:
pilotnet = PilotNet()
torch_input = torch.randn(*batch.shape)
pilotnet.forward(torch_input)
onnx_program = torch.onnx.export(pilotnet, torch_input, 'pilotnet.onnx')

c:\Users\Gordei\miniconda3\envs\nn_proj2\Lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [7]:
pilotnet.forward(batch)

tensor([[0.2197]], grad_fn=<AddmmBackward0>)

In [8]:
perceiver = Perceiver(num_freq_bands=68, depth=4, max_freq=512)
perceiver.forward(batch)

277
torch.Size([1, 204, 538])


RuntimeError: Given normalized_shape=[277], expected input with shape [*, 277], but got input of size[1, 204, 538]